<a href="https://colab.research.google.com/github/RaiahMohamedAmine/Next-Word-Prediction/blob/master/Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow.keras.utils as ku 

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")

--2020-07-23 21:47:57--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-07-23 21:47:57 (81.9 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index )+ 1

input_sentences =[]
for row in corpus :
  token_sentence = tokenizer.texts_to_sequences([row])[0]
  for i in range(1,len(token_sentence)) :
    sentence = token_sentence[:i+1]
    input_sentences.append(sentence)

input_sentences = np.array(input_sentences)
padded = pad_sequences (input_sentences,padding='pre')

predictors, labels = padded[:, :-1], padded[ :, -1]

labels = ku.to_categorical(labels,num_classes=total_words)
print(predictors.shape)
print(labels.shape)

(15462, 10)
(15462, 3211)


In [4]:
input_length = len (predictors[0])
print(total_words)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_words,100,input_length=input_length),
                             tf.keras.layers.LSTM(150),
                           #  tf.keras.layers.Flatten(),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(total_words/2, activation='relu'),
                             tf.keras.layers.Dense(total_words, activation='softmax') 
])

model.compile(loss='categorical_crossentropy', optimizer ="adam", metrics=['acc'])
model.summary()

3211
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
lstm (LSTM)                  (None, 150)               150600    
_________________________________________________________________
dropout (Dropout)            (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 1605)              242355    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 5,870,921
Trainable params: 5,870,921
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(predictors,labels,epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 4s 9ms/step - loss: 6.8059 - acc: 0.0243
Epoch 2/100
484/484 [==============================] - 4s 9ms/step - loss: 6.3941 - acc: 0.0341
Epoch 3/100
484/484 [==============================] - 4s 9ms/step - loss: 6.0945 - acc: 0.0455
Epoch 4/100
484/484 [==============================] - 4s 9ms/step - loss: 5.7217 - acc: 0.0630
Epoch 5/100
484/484 [==============================] - 4s 9ms/step - loss: 5.3374 - acc: 0.0823
Epoch 6/100
484/484 [==============================] - 4s 9ms/step - loss: 4.9512 - acc: 0.1013
Epoch 7/100
484/484 [==============================] - 4s 9ms/step - loss: 4.5173 - acc: 0.1324
Epoch 8/100
484/484 [==============================] - 4s 9ms/step - loss: 4.0322 - acc: 0.1722
Epoch 9/100
484/484 [==============================] - 4s 9ms/step - loss: 3.5093 - acc: 0.2391
Epoch 10/100
484/484 [==============================] - 4s 9ms/step - loss: 3.0092 - acc: 0.3175
Epoch 11/100
484/484 [=================

In [ ]:
total_predictions = 10 
print(tokenizer.word_index)
predict_sentence = 'nassim'
for i in range(1,total_predictions):
  sequence_prediction = tokenizer.texts_to_sequences([predict_sentence])[0]
  padded_prediction = pad_sequences([sequence_prediction],maxlen = input_length, padding='pre')
  prediction = model.predict_classes(padded_prediction)
  predicted_word =""

  for word, index in tokenizer.word_index.items():
    if index == prediction : 
      predicted_word = word
      break
  predict_sentence += " " + predicted_word

print(predict_sentence)